# 外皮情報LV2からLV3へのコンバート

## 入力情報

### 共通

- 地域の区分 (1~8)
- その他の居室の有無 (yes/no)
- 非居室の有無 (yes/no)
- 床面積の合計 (m<sup>2</sup>)
- 主たる居室の床面積 (m<sup>2</sup>)
- その他の居室の床面積 (m<sup>2</sup>)

### 外皮の部位 (一般部位) (開口部) (複数可)

- 名前
- 面積 (m<sup>2</sup>)
- 日射の有無と方位 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 隣接空間の種類 (外気・外気に通じる空間/外気に通じていない空間・外気に通じる床裏/住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏)
- 部位の種類 (一般部位(木造)/一般部位(鉄骨造)/一般部位(鉄筋コンクリート造)/窓等の大部分がガラスで構成されている開口部/ドア等の大部分がガラスで構成されていない開口部)
- 部位の位置(屋根/天井/外壁/床/界壁/界床/天井(界床))
- (断熱性能を表す指標) (単位面積あたり)
- (暖房期の日射熱取得を表す指標) (単位面積あたり)
- (冷房期の日射熱取得を表す指標) (単位面積あたり)

### 外皮の部位 (熱橋等) (複数可)

- 名前
- 面積 (m<sup>2</sup>)
- 日射の有無と方位1 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 日射の有無と方位2 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 隣接空間の種類 (外気・外気に通じる空間/外気に通じていない空間・外気に通じる床裏/住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏)
- 部位の種類 (熱橋/土間床等の外周部)
- (断熱性能を表す指標) (単位長さあたり)
- (暖房期の日射熱取得を表す指標) (単位長さあたり)
- (冷房期の日射熱取得を表す指標) (単位長さあたり)

### 外皮の部位 (土間床等の中央部) (複数可)

- 名前
- 面積
- (断熱性能を表す指標)

## 出力情報

### 共通

- 地域の区分 (1~8)
- その他の居室の有無 (yes/no)
- 非居室の有無 (yes/no)
- 床面積の合計 (m<sup>2</sup>)
- 主たる居室の床面積 (m<sup>2</sup>)
- その他の居室の床面積 (m<sup>2</sup>)

### 外皮の部位 (一般部位) (開口部) (複数可)

- 名前
- 面積 (m<sup>2</sup>)
- 室内側の空間の用途
- 日射の有無と方位 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 隣接空間の種類 (外気・外気に通じる空間/外気に通じていない空間・外気に通じる床裏/住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏)
- 部位の種類 (一般部位(木造)/一般部位(鉄骨造)/一般部位(鉄筋コンクリート造)/窓等の大部分がガラスで構成されている開口部/ドア等の大部分がガラスで構成されていない開口部)
- 部位の位置(屋根/天井/外壁/床/界壁/界床/天井(界床))
- (断熱性能を表す指標) (単位面積あたり)
- (暖房期の日射熱取得を表す指標) (単位面積あたり)
- (冷房期の日射熱取得を表す指標) (単位面積あたり)

### 外皮の部位 (熱橋等) (複数可)

- 名前
- 面積 (m<sup>2</sup>)
- 室内側の空間の用途
- 日射の有無と方位1 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 日射の有無と方位2 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 隣接空間の種類 (外気・外気に通じる空間/外気に通じていない空間・外気に通じる床裏/住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏)
- 部位の種類 (熱橋/土間床等の外周部)
- (断熱性能を表す指標) (単位長さあたり)
- (暖房期の日射熱取得を表す指標) (単位長さあたり)
- (冷房期の日射熱取得を表す指標) (単位長さあたり)

### 外皮の部位 (土間床等の中央部) (複数可)

- 名前
- 面積
- 室内側の空間の用途
- (断熱性能を表す指標)

## 2. 外皮性能計算プログラム＋簡単な追加情報

- 外皮性能計算プログラムの入力情報に、用途別床面積を追加したものより、推定を行う。
- 現状の外皮性能計算プログラムで入力されるのは用途別に分かれていない外皮性能であるため、室用途別の性能を求めるためには、外皮を用途別に割り振る必要がある。
- 住戸全体の各方位の外皮は、室用途別の床面積に応じて各用途に割り当てる。集合住宅の戸境壁等も同様の扱いとする。外皮性能は用途によらず均一に割り付ける。開口部は、2.外皮性能計算プログラムの入力情報より推定を開始する場合は、庇等の日射遮蔽部材を考慮した計算を行うため、方位、遮蔽部材やその寸法等が異なる場合は、別部材として扱う。その他は3.と同様に求める。
 
$$ \qquad
A_{主,i,j}= \frac{A_{i,j}・A_主}{A_主 + A_他 + A_非}, \quad
A_{他,i,j}= \frac{A_{i,j}・A_他}{A_主 + A_他 + A_非}\quad, \quad
A_{非,i,j}= \frac{A_{i,j}・A_主}{A_非 + A_他 + A_非}\quad
\qquad (1),\;(2),\;(3)
\\
$$
$$ \qquad
L_{主,i,j}= \frac{L_{i,j}・A_主}{A_主 + A_他 + A_非}, \quad
L_{他,i,j}= \frac{L_{i,j}・A_他}{A_主 + A_他 + A_非}\quad, \quad
L_{非,i,j}= \frac{L_{i,j}・A_主}{A_非 + A_他 + A_非}\quad
\qquad (4),\;(5),\;(6)
\\
$$
$$ \qquad
U_{主,i,j} =U_{他,i,j} =U_{非,i,j} =U_{i,j} 
\qquad (7)
\\
$$
$$ \qquad
Ψ_{主,i,j} =Ψ_{他,i,j} =Ψ_{非,i,j} =Ψ_{i,j} 
\qquad (8)
\\
$$


- 入力値：

  - 主居室、その他の居室、非居室の床面積$[m ^ 2]$：$A_主$、$A_他$、$A_非$
  - 外皮の方位j、部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,j}$
  - 熱橋及び土間床等の外周部jの長さ$[m]$：$L_{i,j}$
  - 外皮の方位j、部位（一般部位又は開口部）iの熱貫流率$[W/m ^ 2 K]$：$U_{i,j}$
  - 熱橋及び土間床等の外周部jの線熱貫流率$[W/mK]$：$\psi _i$$_,$$_j$
  

- 出力値：

  - 主居室、その他の居室、非居室の外皮の方位j、部位（一般部位又は開口部）iの面積$[m ^ 2]$：$A_{主,i,j}$、$A_{他,i,j}$、$A_{非,i,j}$
  - 主居室、その他の居室、非居室の：熱橋及び土間床等の方位jの外周部iの長さ$[m]$：$L_{主,i,j}$、$L_{他,i,j}$、$L_{非,i,j}$
  - 主居室、その他の居室、非居室の外皮の方位j、部位（一般部位又は開口部）iの熱貫流率$[W/m ^ 2 K]$：$U_{主,i,j}$、$U_{他,i,j}$、$U_{非,i,j}$   
  - 主居室、その他の居室、非居室の熱橋及び土間床等の方位jの外周部iの線熱貫流率$[W/mK]$：$\psi _{主,i,j}$、$\psi _{他,i,j}$、$\psi_{非,i,j}$

In [1]:
""" 室用途別の面積の算出 """
def Calc_Apu(Ap, Au, Aa):
    Apu =  Ap*Au/Aa
    return Apu

print (Calc_Apu(1, 1, 5))

0.2


In [2]:
""" 室用途別の長さの算出 """
def Calc_Lpu(Lp, Au, Aa):
    Lpu =  Lp*Au/Aa
    return Lpu

print (Calc_Lpu(1, 1, 5))

0.2


In [3]:
""" 室用途別の熱貫流率の算出 """
def Calc_Upu(Up):
    Upu =  Up
    return Upu

print (Calc_Upu(1))

1


In [4]:
""" 室用途別の線熱貫流率の算出 """
def Calc_Ψpu(Ψp):
    Ψpu =  Ψp
    return Ψpu

print (Calc_Ψpu(1))

1
